In [1]:
from orbit_generation.visualize import visualize_static_orbits
from orbit_generation.dataset import get_first_period_dataset
from orbit_generation.experiment import get_experiment_parameters
from orbit_generation.propagation import jacobi_constant
from orbit_generation.constants import EM_MU

In [2]:
import numpy as np
import os
import pandas as pd

# Load

In [3]:
data_path = r"/orbit-generation/data/orbits_fix_1500/EM_N_fix_1500.h5"
experiments_folder = r'../experiments'
experiment_folder = r'../experiments/experiment_1'
experiment_id = int(experiment_folder[-1])

In [4]:
images_folder = os.path.join(experiment_folder, 'images')
generated_data_path = os.path.join(experiment_folder, f'exp{experiment_id}_generated_data.npy')
latent_representations_path = os.path.join(experiment_folder, f'exp{experiment_id}_latent_representations.npy')
latent_space_path = os.path.join(images_folder, f'exp{experiment_id}_latent_space')
initial_conditions_file = os.path.join(experiment_folder, f'exp{experiment_id}_corrected_initial_conditions.npy')
generated_data_file = os.path.join(experiment_folder, f'exp{experiment_id}_propagated_corrected_ICs.npy')
converged_orbits_path = os.path.join(images_folder, f'exp{experiment_id}_converged_orbits')
combined_latent_space_path = os.path.join(images_folder, f'exp{experiment_id}_combined_latent_space')
dynamical_orbits_path = os.path.join(images_folder, f'exp{experiment_id}_refined_orbits.html')

# Dataset

In [5]:
params=get_experiment_parameters(experiments_folder=experiments_folder, experiment_id=experiment_id)

data, orbit_df, labels, system_dict = get_first_period_dataset(file_path=data_path, segment_length=params['seq_len'])
data.shape

(45211, 7, 150)

In [6]:
transposed_orbits = np.transpose(data, (0, 2, 1))

if transposed_orbits.shape[2] == 7:
    full_data = transposed_orbits[:,:,1:]
else:
    full_data = transposed_orbits

full_data.shape

(45211, 150, 6)

In [7]:
results = np.array([jacobi_constant(full_data[i][0], EM_MU)[0] for i in range(full_data.shape[0])])
orbit_df['calculated_jacobi'] = results

In [8]:
orbit_df.head()

,id_class,x_0,y_0,x_0,vx_0,vy_0,vz_0,jacobi,period,stability,propagated_periods,calculated_jacobi
0,1.0,0.941000,1.900442e-23,0.509474,2.968938e-13,-0.124968,-3.122717e-12,2.745412,11.555291,211.184678,3.0,2.757415
1,1.0,0.941538,-8.592698e-24,0.508602,2.902340e-13,-0.125672,-3.252212e-12,2.746226,11.551622,210.329145,3.0,2.758229
2,1.0,0.942076,6.956604e-24,0.507729,2.685527e-13,-0.126375,-2.862050e-12,2.747039,11.547936,209.473685,3.0,2.759042
3,1.0,0.942613,1.763946e-24,0.506856,3.153637e-13,-0.127078,-3.791054e-12,2.747850,11.544233,208.618315,3.0,2.759853
4,1.0,0.943150,9.441804e-24,0.505982,2.663322e-13,-0.127781,-2.982720e-12,2.748660,11.540511,207.763051,3.0,2.760663


# Refined Orbits

In [9]:
initial_conditions = np.load(initial_conditions_file)
columns= ['id','old_period', 'new_period', 'iterations_convergence','error','pos_x0','pos_y0','pos_z0','vel_x0','vel_y0','vel_z0']
synthetic_orbit_df = pd.DataFrame(initial_conditions, columns=columns)
generated_data = np.load(generated_data_file)
generated_data.shape

FileNotFoundError: [Errno 2] No such file or directory: '../experiments/experiment_1/exp1_corrected_initial_conditions.npy'

In [15]:
if generated_data.shape[2] == 7:
    display_generation = generated_data[:,:,1:]
else:
    display_generation = generated_data

display_generation.shape

(51, 6, 300)

In [ ]:
new_jacobis = np.array([jacobi_constant(display_generation[i][0], EM_MU)[0] for i in range(display_generation.shape[0])])
synthetic_orbit_df['calculated_jacobi'] = new_jacobis
synthetic_orbit_df.rename(columns={'new_period': 'period'}, inplace=True)

In [ ]:
synthetic_orbit_df.head()

# Compare

In [ ]:
import numpy as np
from scipy.spatial import cKDTree

def calculate_closest_distances(orbit_df, synthetic_orbit_df, features):
    """
    Calculate the distance from each point in synthetic_orbit_df to the closest point in orbit_df
    based on specified features.
    
    Parameters:
    - orbit_df: DataFrame containing the training data.
    - synthetic_orbit_df: DataFrame containing the synthetic data.
    - features: List of features (columns) to use for calculating the distances.
    
    Returns:
    - distances: A NumPy array of the minimum distances from each synthetic point to the nearest orbit point.
    """
    # Extract the relevant features from both DataFrames
    orbit_points = orbit_df[features].values
    synthetic_points = synthetic_orbit_df[features].values

    # Create a KDTree for efficient nearest-neighbor search in orbit_df
    tree = cKDTree(orbit_points)

    # Query the KDTree with the synthetic points to find the distance to the nearest orbit point
    distances, _ = tree.query(synthetic_points, k=1)

    return distances

In [ ]:
# Define the features to be used for distance calculation
features = ['period', 'calculated_jacobi']

# Calculate the distances
distances = calculate_closest_distances(orbit_df, synthetic_orbit_df, features)

In [ ]:
distances

In [ ]:
import matplotlib.pyplot as plt

def plot_comparison(orbit_df, synthetic_orbit_df):
    """
    Function to create a scatter plot comparing 'period' and 'calculated_jacobi'
    between two DataFrames and plot the index of points in synthetic_orbit_df.
    
    Parameters:
    - orbit_df: DataFrame containing 'period' and 'calculated_jacobi' columns
    - synthetic_orbit_df: DataFrame containing 'period' and 'calculated_jacobi' columns
    """
    plt.figure(figsize=(10, 6))

    # Plot for orbit_df
    plt.scatter(orbit_df['period'], orbit_df['calculated_jacobi'], color='blue', label='orbit_df', marker='o')

    # Plot for synthetic_orbit_df
    plt.scatter(synthetic_orbit_df['period'], synthetic_orbit_df['calculated_jacobi'], color='red', label='synthetic_orbit_df', marker='x')

    # Annotate the points in synthetic_orbit_df with their indices
    for i in synthetic_orbit_df.index:
        plt.text(synthetic_orbit_df['period'].loc[i], 
                 synthetic_orbit_df['calculated_jacobi'].loc[i], 
                 str(int(i)), 
                 fontsize=9, color='black', ha='right')

    # Add labels and title
    plt.xlabel('Period')
    plt.ylabel('Calculated Jacobi')
    plt.title('Comparison of Features Between Two DataFrames')
    plt.legend()  # Add legend to distinguish between the groups

    # Display the plot
    plt.grid(True)
    plt.show()

In [ ]:
plot_comparison(orbit_df, synthetic_orbit_df)

In [ ]:
visualize_static_orbits(np.transpose(display_generation, (0, 2, 1)), orbit_indices=[9,26,35])